In [2]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [149]:
import dgl
import pandas as pd
import numpy as np
import torch as th
import re
import ast
import pickle
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_excel('Оквэды с инфо.xlsx', index_col=0)
data.head()

,ОКВЭД,Альтернативный ОКВЭД,info
0,01.1,01.1,"Сельское, лесное хозяйство, охота, рыболовство..."
1,01.11,01.11,"Сельское, лесное хозяйство, охота, рыболовство..."
2,01.11.1,01.11.1,"Сельское, лесное хозяйство, охота, рыболовство..."
3,01.11.11,01.11.11,"Сельское, лесное хозяйство, охота, рыболовство..."
4,01.11.12,01.11.12,"Сельское, лесное хозяйство, охота, рыболовство..."


In [ ]:
okveds = data['ОКВЭД'].values

first = np.array([])
second = np.array([])
third = np.array([])

for i in okveds:
    first = np.append(first, i.split('.')[0])
    second = np.append(second, '.'.join(i.split('.')[:2]))
    if len(i) > 5:
        third = np.append(third, i)

In [ ]:
first = np.unique(first)
second = np.unique(second)
third = np.unique(third)

In [ ]:
dict_edges = {}

for f in first:
    dict_edges[f] = {}

for s in second:
    dict_edges[s[:2]][s] = []

for t in third:
    f = t[:2]
    s = '.'.join(t.split('.')[:2])
    dict_edges[f][s].append(t)

In [ ]:
dict_edges

{'01': {'01.1': [],
  '01.11': ['01.11.1',
   '01.11.11',
   '01.11.12',
   '01.11.13',
   '01.11.14',
   '01.11.15',
   '01.11.16',
   '01.11.19',
   '01.11.2',
   '01.11.3',
   '01.11.31',
   '01.11.32',
   '01.11.33',
   '01.11.39'],
  '01.12': [],
  '01.13': ['01.13.1',
   '01.13.11',
   '01.13.12',
   '01.13.2',
   '01.13.3',
   '01.13.31',
   '01.13.39',
   '01.13.4',
   '01.13.5',
   '01.13.51',
   '01.13.52',
   '01.13.6',
   '01.13.9'],
  '01.14': [],
  '01.15': [],
  '01.16': ['01.16.2'],
  '01.19': ['01.19.1', '01.19.2', '01.19.21', '01.19.3', '01.19.9'],
  '01.2': [],
  '01.21': [],
  '01.22': [],
  '01.23': [],
  '01.24': [],
  '01.25': ['01.25.1', '01.25.2', '01.25.3'],
  '01.26': [],
  '01.27': [],
  '01.28': ['01.28.1'],
  '01.29': [],
  '01.30': [],
  '01.4': [],
  '01.41': ['01.41.1', '01.41.11', '01.41.12', '01.41.2', '01.41.21'],
  '01.42': ['01.42.1', '01.42.11', '01.42.12'],
  '01.43': ['01.43.1'],
  '01.45': ['01.45.1', '01.45.4'],
  '01.46': ['01.46.1', '01.46.1

In [ ]:
first_nodes = []
second_nodes = []

for k, v in dict_edges.items():
    first_nodes += [k] * len(v)
    second_nodes += list(v.keys())

for val in dict_edges.values():
    for k, v in val.items():
        if len(val[k]) > 0:
            first_nodes += [k] * len(v)
            second_nodes += v

In [ ]:
data_edges_ok_ok = pd.DataFrame([first_nodes, second_nodes]).T.rename(columns={0 : 'First', 1 : 'Second'})
data_edges_ok_ok.head()

,First,Second
0,01,01.1
1,01,01.11
2,01,01.12
3,01,01.13
4,01,01.14


In [ ]:
data_edges_ok_ok.tail()

,First,Second
1402,93.29,93.29.9
1403,95.22,95.22.1
1404,95.24,95.24.1
1405,95.29,95.29.9
1406,96.02,96.02.2


In [ ]:
data_edges_ok_ok.shape

(1407, 2)

In [ ]:
data_edges_ok_ok.to_excel('Edges_okveds.xlsx')

In [96]:
data = pd.read_excel('Датасет.xlsx', index_col=0)

new_name = [data.iloc[i, 0] + str(i) for i in range(len(data))]
data['Наименование / ФИО'] = new_name

In [98]:
data_edges_comp_main = pd.DataFrame(data['Наименование / ФИО']).rename(columns={'Наименование / ФИО' : 'First'})
data_edges_comp_main['Second'] = data['Основной вид деятельности'].apply(lambda x: x.split()[0])
data_edges_comp_main.head()

,First,Second
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.11.1
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОНОВА""1",01.50
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОПРОМЫШЛЕННАЯ КОМПАНИ...",01.2
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АГРОФИРМА КУХАРИВСКАЯ""3",01.41.11
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""БЕЛАГРО""4",01.50


In [99]:
data_edges_comp_main.shape

(1485, 2)

In [ ]:
data_edges_comp_main.to_excel('Edges_comp_main.xlsx')

In [100]:
dop = [ast.literal_eval(s) for s in data['Дополнительный вид деятельности']]

first_nodes = []
second_nodes = []

for i in range(len(dop)):
    first_nodes += [data['Наименование / ФИО'][i]] * len(dop[i])
    second_nodes += dop[i]

data_edges_comp_dop = pd.DataFrame([first_nodes, second_nodes]).T.rename(columns={0 : 'First', 1 : 'Second'})
data_edges_comp_dop.head()

,First,Second
0,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.11.2
1,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.11.3
2,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.13
3,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.13.1
4,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""50 ЛЕТ ОКТЯБРЯ""0",01.13.51


In [101]:
data_edges_comp_dop.tail()

,First,Second
31662,"САНАТОРИЙ "" ВИТА "" ( ОБЩЕСТВО С ОГРАНИЧЕННОЙ О...",96.04
31663,"СТОМАТОЛОГИЧЕСКАЯ КЛИНИКА "" ВИЗИТ "" ОБЩЕСТВО С...",86.21
31664,"СТОМАТОЛОГИЧЕСКАЯ КЛИНИКА "" ВИЗИТ "" ОБЩЕСТВО С...",86.23
31665,"СТОМАТОЛОГИЧЕСКАЯ КЛИНИКА "" ВИЗИТ "" ОБЩЕСТВО С...",86.90.4
31666,"СТОМАТОЛОГИЧЕСКАЯ КЛИНИКА "" ВИЗИТ "" ОБЩЕСТВО С...",86.90.9


In [102]:
data_edges_comp_dop.shape

(31667, 2)

In [ ]:
data_edges_comp_dop.to_excel('Edges_comp_dop.xlsx')

In [61]:
okveds_info = pd.read_excel('/content/Оквэды с инфо.xlsx', index_col=0, usecols=[0, 1, 3])
company = pd.read_excel('/content/Датасет.xlsx', index_col=0)
okv_okv = pd.read_excel('/content/Edges_okveds.xlsx', index_col=0, converters={'First':str, 'Second':str})
comp_main = pd.read_excel('/content/Edges_comp_main.xlsx', index_col=0, converters={'First':str, 'Second':str})
comp_dop = pd.read_excel('/content/Edges_comp_dop.xlsx', index_col=0, converters={'First':str, 'Second':str})

In [103]:
company = data
comp_main = data_edges_comp_main
comp_dop = data_edges_comp_dop

In [107]:
lst_ok = list(set(np.hstack((okv_okv['First'].values, okv_okv['Second'].values))) - set(okveds_info['ОКВЭД'].values))
all_okv = np.hstack((okveds_info['ОКВЭД'].values, np.array(lst_ok)))

In [108]:
le_okved = LabelEncoder()
le_company = LabelEncoder()

In [109]:
le_okved.fit(all_okv)

LabelEncoder()

In [110]:
le_company.fit(company['Наименование / ФИО'])

LabelEncoder()

In [111]:
first_ok = th.tensor(le_okved.transform(okv_okv['First']))
second_ok = th.tensor(le_okved.transform(okv_okv['Second']))

In [112]:
first_comp_main = th.tensor(le_company.transform(comp_main['First']))
second_comp_main = th.tensor(le_okved.transform(comp_main['Second']))

In [113]:
first_comp_dop = th.tensor(le_company.transform(comp_dop['First']))
second_comp_dop = th.tensor(le_okved.transform(comp_dop['Second']))

In [114]:
G = dgl.heterograph({
   ('okved', 'parent', 'okved'): (first_ok, second_ok),
   ('company', 'main_okved', 'okved'): (first_comp_main, second_comp_main),
   ('company', 'dop_okved', 'okved'): (first_comp_dop, second_comp_dop)
})

In [115]:
G.nodes('okved')

tensor([   0,    1,    2,  ..., 1488, 1489, 1490])

In [116]:
ebm_okved = np.load('/content/Embeddings_okved.npy')
ebm_okved = th.tensor(np.vstack((ebm_okved, np.zeros((95, 768)))))
ebm_okved

tensor([[ 0.1231, -0.5409,  0.8120,  ...,  0.6631,  0.6793,  0.3004],
        [-0.0747, -0.2076,  0.1517,  ...,  0.6889,  0.2814, -0.3052],
        [-0.7036, -0.7664, -0.2374,  ...,  0.9325,  0.6944,  0.4097],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [124]:
ebm_address = np.load('/content/Embeddings_address.npy')
ebm_address = th.tensor(ebm_address)
ebm_address

tensor([[ 0.1039,  0.6427,  0.3033,  ...,  0.7175,  0.2035,  0.4756],
        [ 0.6728, -0.0102,  0.3042,  ...,  0.8129, -0.1617,  0.0835],
        [ 0.2859,  0.1696,  0.1855,  ...,  0.1728,  0.1623, -0.0566],
        ...,
        [ 0.3457, -0.0753,  0.5041,  ...,  1.3337,  0.2449,  0.2130],
        [ 0.5919, -0.1035,  1.1670,  ...,  1.1321,  0.0218,  0.3228],
        [ 0.5919, -0.1035,  1.1670,  ...,  1.1321,  0.0218,  0.3228]],
       dtype=torch.float64)

In [130]:
G.nodes['okved'].data['embeddings'] = ebm_okved
G.nodes['company'].data['category'] = th.tensor(company['Категория'])
G.nodes['company'].data['licenses'] = th.tensor(company['Наличие лицензий'])
G.nodes['company'].data['is_social'] = th.tensor(company['Является социальным предприятием'])
G.nodes['company'].data['workers'] = th.tensor(company['Среднесписочная численность работников за предшествующий календарный год'])
G.nodes['company'].data['address'] = ebm_address

In [140]:
dgl.save_graphs('Graph.bin', G)

In [150]:
pickle.dump(le_okved, open('le_okved.sav', 'wb'))
pickle.dump(le_company, open('le_company.sav', 'wb'))